# How to load CSV files from stage to Snowflake Notebooks 📁

In this example, we will show how you can load a CSV file from stage and create a table with Snowpark. 

First, let's use the `get_active_session` command to get the [session](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.Session#snowflake.snowpark.Session) context variable to work with Snowpark as follows:

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()
# Add a query tag to the session. This helps with troubleshooting and performance monitoring.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"notebook_demo_pack", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook", "vignette":"csv_from_s3"}}
print(session)

Next, we will create an [external stage](https://docs.snowflake.com/en/sql-reference/sql/create-stage) that references data files stored in a location outside of Snowflake, in this case, the data lives in a [S3 bucket](https://docs.snowflake.com/en/user-guide/data-load-s3-create-stage).

In [ ]:
CREATE STAGE IF NOT EXISTS TASTYBYTE_STAGE 
	URL = 's3://sfquickstarts/frostbyte_tastybytes/';

Let's take a look at the files in the stage.

In [ ]:
LS @TASTYBYTE_STAGE/app/app_orders;

We can use [Snowpark DataFrameReader](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/1.14.0/api/snowflake.snowpark.DataFrameReader) to read in the CSV file.

By using the `infer_schema = True` option, Snowflake will automatically infer the schema based on data types present in CSV file, so that you don't need to specify the schema beforehand. 

In [ ]:
# Create a DataFrame that is configured to load data from the CSV file.
df = session.read.options({"infer_schema":True}).csv('@TASTYBYTE_STAGE/app/app_orders/app_order_detail.csv.gz')

In [ ]:
df

Now that the data is loaded into a Snowpark DataFrame, we can work with the data using [Snowpark DataFrame API](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/api/snowflake.snowpark.DataFrame). 

For example, I can compute descriptive statistics on the columns.

In [ ]:
df.describe()

We can write the dataframe into a table called `APP_ORDER` and query it with SQL. 

In [ ]:
df.write.mode("overwrite").save_as_table("APP_ORDER")

In [ ]:
-- Preview the newly created APP_ORDER table
SELECT * from APP_ORDER;

Finally, we show how you can read the table back to Snowpark via the `session.table` syntax.

In [ ]:
df = session.table("APP_ORDER")
df

From here, you can continue to query and process the data. 

In [ ]:
df.groupBy('"c4"').count()

In [ ]:
-- Teardown table and stage created as part of this example
DROP TABLE APP_ORDER;
DROP STAGE TASTYBYTE_STAGE;

### Conclusion
In this example, we took a look at how you can load a CSV file from an external stage to process and query the data in your notebook using Snowpark. You can learn more about how to work with your data using Snowpark Python [here](https://docs.snowflake.com/en/developer-guide/snowpark/python/index).